# ETL 

Iniciamos el proceso de ETL con el dataset suministrado por el Observatorio de Movilidad y Seguridad Vial que incluye informacion acerca de accidentes de transito, concretamente consta de dos hojas dentro de un archivo excel: una que presenta informacion sobre los hechos del accidente y otra que presenta informacion sobre las victimas involucradas en dichos accidentes.

In [375]:
# importamos librerias
import pandas as pd 
import numpy as np 
import warnings 
warnings.filterwarnings('ignore')

cargamos el archivo excel y leemos las hojas que contiene

In [376]:
# procedemos a leer las hojas dentro del archivo excel
df_homicidios= pd.ExcelFile('data/homicidios.xlsx')
df_hechos= pd.read_excel(df_homicidios, 'HECHOS')
df_victimas= pd.read_excel(df_homicidios, 'VICTIMAS')

# df_hechos

In [377]:
# vista general del dataframe 'hechos'
df_hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [378]:
# informacion general del dataframe
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [379]:
# revisamos las dimensiones de nuestro dataframe
dimensiones_hecho=df_hechos.shape
print(f'las dimensiones de nuestro dataframe son: {dimensiones_hecho}')

las dimensiones de nuestro dataframe son: (696, 21)


chequeo de valores duplicados en el dataset

In [380]:
# chequeamos valores duplicados
registros_duplicados=df_hechos.duplicated().sum()
print(f'cantidad de registros duplicados: {registros_duplicados}')

cantidad de registros duplicados: 0


podemos verificar que el dataframe no contiene valores duplicados

In [381]:
# chequeamos valores nulos
df_hechos.isnull().sum()


ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

podemos observar que la columna 'Altura' tiene una gran cantidad de valores nulos, procederemos a sacar la porcentaje de  sus valores nulos con el proposito de saber si procedemos a eliminarla o no

In [382]:
# proporcion de valores nulos de la columna 'Altura'
porcentaje_nulos_altura=df_hechos['Altura'].isnull().sum() / len(df_hechos)
print(f'el porcentaje de valores nulos de la columna Altura es: {porcentaje_nulos_altura:.2%}')

el porcentaje de valores nulos de la columna Altura es: 81.47%


ya que la porcentaje de sus valores nulos es del 81% (muy alto), procedemos a eliminar la columna 'Altura'

In [383]:
# eliminamos columna 'Altura'
df_hechos= df_hechos.drop(columns=['Altura'], axis=1)

In [384]:
df_hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


en esta tabla de 'hechos' encontramos ciertas columnas que contienen informacion que ya se encuentra agrupada en otras columnas. por esta razon, oprtamos por eliminarlas para simplificar y optimizar la estructura del dataset

In [385]:
df_hechos=df_hechos.drop(columns={'AAAA', 'MM','DD', 'Cruce', 'Calle', 'Dirección Normalizada', 'XY (CABA)'})
df_hechos.head()

,ID,N_VICTIMAS,FECHA,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,9,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,1,-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,8,-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,1,-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [386]:
# analizamos columna 'HH'
df_hechos['HH'].unique()

array([4, 1, 7, 0, 5, 18, 19, 15, 11, 22, 16, 9, 23, 6, 10, 17, 12, 8, 20,
       21, 14, 3, 2, 13, 'SD'], dtype=object)

podemos observar que en esta columna tenemos un campo etiquetado como 'SD' , se contemplaron algunas medidas para reemplazar su valor como la opcion de calcular la mediana de las horas y reemplazarlo con su resultado, pero no seria la mejor opcion ya que su resultado podria ser no representativo en el contexto de nuestro caso.
asi que decidimos mantener el campo 'SD'

In [387]:
# chequeamos valores nulos
df_hechos.isnull().sum()

ID                 0
N_VICTIMAS         0
FECHA              0
HORA               0
HH                 0
LUGAR_DEL_HECHO    0
TIPO_DE_CALLE      0
COMUNA             0
pos x              0
pos y              0
PARTICIPANTES      0
VICTIMA            0
ACUSADO            0
dtype: int64

In [388]:
# informacion general del dataframe
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ID               696 non-null    object        
 1   N_VICTIMAS       696 non-null    int64         
 2   FECHA            696 non-null    datetime64[ns]
 3   HORA             696 non-null    object        
 4   HH               696 non-null    object        
 5   LUGAR_DEL_HECHO  696 non-null    object        
 6   TIPO_DE_CALLE    696 non-null    object        
 7   COMUNA           696 non-null    int64         
 8   pos x            696 non-null    object        
 9   pos y            696 non-null    object        
 10  PARTICIPANTES    696 non-null    object        
 11  VICTIMA          696 non-null    object        
 12  ACUSADO          696 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(10)
memory usage: 70.8+ KB


observamos que las columnas 'pos x' y 'pos y', que representan un sistema de coordenadas, están actualmente clasificadas como tipo de dato object. Para mejorar la representación, planeamos modificar el tipo de dato a float, tambien seran reenombradas con un nombre que represente mejor su funcion

In [389]:
# filtramos las filas de las columnas con posiciones no validas
posiciones_valid = ~(df_hechos['pos x'].str.contains('[^0-9.,-]') | df_hechos['pos y'].str.contains('[^0-9.,-]') | (df_hechos['pos x'] == '.') | (df_hechos['pos y'] == '.'))

# establecemos las posiciones no validas como valores NaN
df_hechos.loc[~posiciones_valid, ['pos x', 'pos y']] = np.nan

# Convertimos las cadenas a tipo numérico (float)
df_hechos['pos x'] = df_hechos['pos x'].str.replace(',', '.').astype(float)
df_hechos['pos y'] = df_hechos['pos y'].str.replace(',', '.').astype(float)

# reenombramos columnas
df_hechos= df_hechos.rename(columns={'pos x':'COORDENADAS_X', 'pos y':'COORDENADAS_Y'})

# df_victimas

In [390]:
df_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [391]:
# informacion general del dataframe
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [392]:
# chequeamos registros duplicados
valores_duplicados= df_victimas.duplicated().sum()
print(f'cantidad de registros duplicados {valores_duplicados}')

cantidad de registros duplicados 0


comprobamos que el dataframe no presenta registros duplicados

In [393]:
# chequeamos nulos
df_victimas.isnull().sum()

ID_hecho               0
FECHA                  0
AAAA                   0
MM                     0
DD                     0
ROL                    0
VICTIMA                0
SEXO                   0
EDAD                   0
FECHA_FALLECIMIENTO    0
dtype: int64

In [394]:
# renombramos columna 'AAAA' a 'AÑO'
df_victimas= df_victimas.rename(columns={'AAAA':'AÑO'})
df_victimas.head(2)

,ID_hecho,FECHA,AÑO,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00


podemos darnos cuenta que las columnas 'FECHA', 'MM' y 'DD' contienen informacion que ya se encuentra agrupada en la columna 'FECHA_FALLECIMIENTO' por lo que optamos por eliminarlas para mejorar para optimizar la estructura de nuestro dataset

In [395]:
# eliminamos columnas
df_victimas= df_victimas.drop(columns=['FECHA', 'MM', 'DD'])

In [396]:
df_victimas.head(2)

,ID_hecho,AÑO,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00


# procedemos agrupar ambas hojas, ya que por la informacion que contienen resulta conveniente agruparlas en un solo dataframe con el cual haremos el procedimiento del EDA mas adelante

In [397]:
df_homicidios= pd.merge(df_hechos, df_victimas, left_on= 'ID', right_on='ID_hecho')
df_homicidios.head()

,ID,N_VICTIMAS,FECHA,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,COORDENADAS_X,COORDENADAS_Y,PARTICIPANTES,VICTIMA_x,ACUSADO,ID_hecho,AÑO,ROL,VICTIMA_y,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,2016-0001,2016,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,1,2016-01-02,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,9,-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,2016-0002,2016,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,1,2016-01-03,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,1,-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,2016-0003,2016,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,1,2016-01-10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,8,-58.465039,-34.680930,MOTO-SD,MOTO,SD,2016-0004,2016,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,1,2016-01-21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,1,-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,2016-0005,2016,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


en esta etapa observamos que los valores nulos estan representados como 'SD' lo que hace alusion a 'Sin Dato', con el proposito de evitar sesgos en la etapa de visualizacion, procederemos a sustituir estos valores por nulos

In [398]:
# aplicamos funcion que reemplazara dichos valores
def reemplazar_valores_SD(df):
    for i in range(len(df)):
        for col in df.columns:
            if df.loc[i, col] == "SD":
                df.loc[i, col] = pd.NaT
    return df

reemplazar_valores_SD(df_homicidios)

,ID,N_VICTIMAS,FECHA,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,COORDENADAS_X,COORDENADAS_Y,PARTICIPANTES,VICTIMA_x,ACUSADO,ID_hecho,AÑO,ROL,VICTIMA_y,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,2016-0001,2016,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,1,2016-01-02,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,9,-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,2016-0002,2016,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,1,2016-01-03,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,1,-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,2016-0003,2016,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,1,2016-01-10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,8,-58.465039,-34.680930,MOTO-SD,MOTO,NaT,2016-0004,2016,CONDUCTOR,MOTO,MASCULINO,18,NaT
4,2016-0005,1,2016-01-21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,1,-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,2016-0005,2016,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0093,1,2021-12-13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,7,-58.433538,-34.645616,MOTO-AUTO,MOTO,AUTO,2021-0093,2021,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18 00:00:00
713,2021-0094,1,2021-12-20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,9,-58.467398,-34.651178,MOTO-AUTO,MOTO,AUTO,2021-0094,2021,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2021-12-20 00:00:00
714,2021-0095,1,2021-12-30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,11,-58.472934,-34.619847,MOTO-CARGAS,MOTO,CARGAS,2021-0095,2021,CONDUCTOR,MOTO,MASCULINO,27,2022-01-02 00:00:00
715,2021-0096,1,2021-12-15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,9,-58.470668,-34.650217,AUTO-CARGAS,AUTO,CARGAS,2021-0096,2021,CONDUCTOR,AUTO,MASCULINO,60,2021-12-20 00:00:00


observamos que algunas columnas tienen informacion que ya se encuentra agrupada en otras columnas, asi que procedemos a eliminarlas

In [399]:
df_homicidios= df_homicidios.drop(columns=['HH', 'ID_hecho', 'VICTIMA_y'])
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   717 non-null    object        
 1   N_VICTIMAS           717 non-null    int64         
 2   FECHA                717 non-null    datetime64[ns]
 3   HORA                 716 non-null    object        
 4   LUGAR_DEL_HECHO      716 non-null    object        
 5   TIPO_DE_CALLE        717 non-null    object        
 6   COMUNA               717 non-null    int64         
 7   COORDENADAS_X        704 non-null    float64       
 8   COORDENADAS_Y        704 non-null    float64       
 9   PARTICIPANTES        717 non-null    object        
 10  VICTIMA_x            708 non-null    object        
 11  ACUSADO              694 non-null    object        
 12  AÑO                  717 non-null    int64         
 13  ROL                  706 non-null  

In [400]:
# chequeamos valores nulos
df_homicidios.isnull().sum()

ID                      0
N_VICTIMAS              0
FECHA                   0
HORA                    1
LUGAR_DEL_HECHO         1
TIPO_DE_CALLE           0
COMUNA                  0
COORDENADAS_X          13
COORDENADAS_Y          13
PARTICIPANTES           0
VICTIMA_x               9
ACUSADO                23
AÑO                     0
ROL                    11
SEXO                    6
EDAD                   53
FECHA_FALLECIMIENTO    68
dtype: int64

los valores nulos de las columnas 'VICTIMA_x', 'ACUSADO', 'SEXO' y 'ROL' le asignaremos el valor 'NO REGISTRADO' para poder conservar esta cantidad de registros que contienen informacion estadistica importante

In [401]:
# reemplazamos valores nulos por 'NO REGISTRADO'
df_homicidios['VICTIMA_x'].fillna('NO IDENTIFICADO', inplace=True)
df_homicidios['ACUSADO'].fillna('NO IDENTIFICADO', inplace=True)
df_homicidios['SEXO'].fillna('NO IDENTIFICADO', inplace=True)
df_homicidios['ROL'].fillna('NO IDENTIFICADO', inplace=True)

In [402]:
# chequeo de valores nulos
df_homicidios.isnull().sum()

ID                      0
N_VICTIMAS              0
FECHA                   0
HORA                    1
LUGAR_DEL_HECHO         1
TIPO_DE_CALLE           0
COMUNA                  0
COORDENADAS_X          13
COORDENADAS_Y          13
PARTICIPANTES           0
VICTIMA_x               0
ACUSADO                 0
AÑO                     0
ROL                     0
SEXO                    0
EDAD                   53
FECHA_FALLECIMIENTO    68
dtype: int64

las columnas 'HORA' y 'LUGAR_DEL_HECHO' solo contienen 2 registros nulos para un total de 717 registros en nuestro dataframe, asi que procederemos a eliminar estos 2 valores nulos

In [403]:
df_homicidios.dropna(subset=['HORA'], inplace=True)
df_homicidios.dropna(subset=['LUGAR_DEL_HECHO'],inplace=True)

la columna 'EDAD' tiene un total de 53 valores nulos, son muchos registros que no queremos perder, asi que procederemos a aplicar la media de todas las edades presentes para cubrir esos valores nulos

In [404]:
# primero convertimos los valores de la columna a tipo de dato numerico
df_homicidios['EDAD']= pd.to_numeric(df_homicidios['EDAD'], errors='coerce')
# calculamos la media de los valores
media_edades=df_homicidios['EDAD'].dropna().mean()
# imputamos la media en los valores nulos
df_homicidios['EDAD'].fillna(media_edades, inplace=True)
# cambiamos tipo de dato a int 
df_homicidios['EDAD']= df_homicidios['EDAD'].astype(int)

la columna 'FECHA_FALLECIMIENTO' presenta un total de 68 valores nulos que seguramente tienen informacion que nos servira mucho en nuestro proceso de EDA, asi que procederemos a rellenar esos valores nulos los de la columna 'FECHA'

In [405]:
# rellenamos valores nulos
df_homicidios['FECHA_FALLECIMIENTO'] = df_homicidios['FECHA_FALLECIMIENTO'].fillna(df_homicidios['FECHA'])

# cambiamos tipo de dato de columna a datetime
df_homicidios['FECHA_FALLECIMIENTO']= pd.to_datetime(df_homicidios['FECHA_FALLECIMIENTO'])

In [406]:
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   COORDENADAS_X        703 non-null    float64       
 8   COORDENADAS_Y        703 non-null    float64       
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA_x            715 non-null    object        
 11  ACUSADO              715 non-null    object        
 12  AÑO                  715 non-null    int64         
 13  ROL                  715 non-null    obj

In [407]:
df_homicidios.head(2)

,ID,N_VICTIMAS,FECHA,HORA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,COORDENADAS_X,COORDENADAS_Y,PARTICIPANTES,VICTIMA_x,ACUSADO,AÑO,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,04:00:00,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,2016,CONDUCTOR,MASCULINO,19,2016-01-01
1,2016-0002,1,2016-01-02,01:15:00,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,9,-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,2016,CONDUCTOR,MASCULINO,70,2016-01-02


procedemos a organizar la estructura de las direcciones de la columna 'LUGAR_DEL_HECHO'

In [408]:
# reemplazamos las comas de la columna por espacios vacios, para despues reorganizarlas
df_homicidios['LUGAR_DEL_HECHO']= df_homicidios['LUGAR_DEL_HECHO'].str.replace(',', '')

In [409]:
# definimos funcion para organizar la estructura de los valores de la columna
def reorganizar_direcc(address):
    words = address.split()
    if "Av." in words:
        words.remove("Av.")
        words = ["Av."] + words
    return " ".join(words)

# Aplicamos la función a la columna y visualizamos
df_homicidios["LUGAR_DEL_HECHO"] = df_homicidios["LUGAR_DEL_HECHO"].apply(reorganizar_direcc)
df_homicidios.head(2)

,ID,N_VICTIMAS,FECHA,HORA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,COORDENADAS_X,COORDENADAS_Y,PARTICIPANTES,VICTIMA_x,ACUSADO,AÑO,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,04:00:00,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,2016,CONDUCTOR,MASCULINO,19,2016-01-01
1,2016-0002,1,2016-01-02,01:15:00,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,9,-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,2016,CONDUCTOR,MASCULINO,70,2016-01-02


procedemos a estandarizar la columna 'HORA'

In [410]:
# reemplazamos todos los valores que tengan el formato incorrecto
formato_incorrecto = pd.to_datetime(df_homicidios["HORA"], errors="coerce").isnull()
df_homicidios.loc[~formato_incorrecto, "HORA"] = pd.to_datetime(df_homicidios.loc[~formato_incorrecto, "HORA"]).dt.strftime('%H:%M:%S')
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   COORDENADAS_X        703 non-null    float64       
 8   COORDENADAS_Y        703 non-null    float64       
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA_x            715 non-null    object        
 11  ACUSADO              715 non-null    object        
 12  AÑO                  715 non-null    int64         
 13  ROL                  715 non-null    obj

In [411]:
# renombramos columna 'VICTIMA_x' a 'VICTIMA'
df_homicidios=df_homicidios.rename(columns={'VICTIMA_x':'VICTIMA'})
df_homicidios.head(2)

,ID,N_VICTIMAS,FECHA,HORA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,COORDENADAS_X,COORDENADAS_Y,PARTICIPANTES,VICTIMA,ACUSADO,AÑO,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,04:00:00,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,2016,CONDUCTOR,MASCULINO,19,2016-01-01
1,2016-0002,1,2016-01-02,01:15:00,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,9,-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,2016,CONDUCTOR,MASCULINO,70,2016-01-02


# KPI 1

Empezamos a construir toda la data necesaria para nuestro primer KPI


Para calcular el primer KPI necesitamos obtener la cantidad de muertes correspondientes a cada semestre de los años registrados, asi que creamos la columna 'SEMESTRE' para hacer mas sencillo el calculo

In [412]:
df_homicidios["SEMESTRE"] = [1 if month <= 6 else 2 for month in df_homicidios["FECHA"].dt.month]
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   COORDENADAS_X        703 non-null    float64       
 8   COORDENADAS_Y        703 non-null    float64       
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA              715 non-null    object        
 11  ACUSADO              715 non-null    object        
 12  AÑO                  715 non-null    int64         
 13  ROL                  715 non-null    obj

In [413]:
# agrupamos por año y por semestre para determinar la cantidad de victimas
df_homicidios_suma= df_homicidios.groupby(['AÑO','SEMESTRE'])['N_VICTIMAS'].sum().reset_index()
df_homicidios_suma

,AÑO,SEMESTRE,N_VICTIMAS
0,2016,1,67
1,2016,2,82
2,2017,1,81
3,2017,2,79
4,2018,1,76
5,2018,2,85
6,2019,1,59
7,2019,2,46
8,2020,1,31
9,2020,2,56


# cargamos dataset poblacionCABA.csv

todo el tratamiento inicial que se le hizo a este dataset se encuentra en el archivo [Web_Scraping.ipynb](Web_Scraping.ipynb)

In [414]:
df_poblacion= pd.read_csv('data/poblacionCABA.csv')


In [415]:
# eliminamos las columnas que no vamos a necesitar
df_poblacion= df_poblacion.drop(columns=['Unnamed: 0','Poblacion 2010','Crecimiento 2010-2022'])
df_poblacion

,Comuna,Poblacion 2016,Poblacion 2017,Poblacion 2018,Poblacion 2019,Poblacion 2020,Poblacion 2021
0,13,247686,250522,253391,256292,259227,262195
1,14,237203,239129,241071,243028,245001,246990
2,12,217446,220476,223550,226666,229825,233028
3,4,224495,225554,226618,227687,228761,229840
4,1,214398,215850,217313,218785,220267,221760
5,7,218707,218395,218083,217772,217461,217151
6,8,195839,197311,198793,200287,201793,203309
7,6,189418,191738,194087,196464,198871,201307
8,11,196538,197678,198825,199979,201139,202306
9,15,190078,191359,192648,193946,195252,196567


In [416]:
df_poblacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Comuna          15 non-null     int64
 1   Poblacion 2016  15 non-null     int64
 2   Poblacion 2017  15 non-null     int64
 3   Poblacion 2018  15 non-null     int64
 4   Poblacion 2019  15 non-null     int64
 5   Poblacion 2020  15 non-null     int64
 6   Poblacion 2021  15 non-null     int64
dtypes: int64(7)
memory usage: 972.0 bytes


In [417]:
# generamos un dataframe con la poblacion actualizada
poblacion_suma= df_poblacion[['Poblacion 2016', 'Poblacion 2017', 'Poblacion 2018', 'Poblacion 2019', 'Poblacion 2020','Poblacion 2021']].sum()
df_poblacion_suma= poblacion_suma.to_frame().reset_index()

# definimos columnas de nuestro dataframe
df_poblacion_suma.columns= ["AÑO", "POBLACION_TOTAL"]

# eliminamos la palabra 'Poblacion' de los valores
df_poblacion_suma['AÑO']= df_poblacion_suma['AÑO'].str.replace('Poblacion', '', case=False)
df_poblacion_suma

,AÑO,POBLACION_TOTAL
0,2016,3002178
1,2017,3021444
2,2018,3040889
3,2019,3060508
4,2020,3080306
5,2021,3100282


procedemos a hacer un merged entre el dataframe con los semestres y el dataframe con la poblacion anualizada

In [418]:
# primero debemos convertir el tipo de dato de la columna 'AÑO' de cada dataframe a int
df_homicidios_suma['AÑO']= df_homicidios_suma['AÑO'].astype('int64')
df_poblacion_suma['AÑO']= df_poblacion_suma['AÑO'].astype('int64')

# hacemos el merged
data_kpi01=df_homicidios_suma.merge(df_poblacion_suma, on='AÑO')
data_kpi01

,AÑO,SEMESTRE,N_VICTIMAS,POBLACION_TOTAL
0,2016,1,67,3002178
1,2016,2,82,3002178
2,2017,1,81,3021444
3,2017,2,79,3021444
4,2018,1,76,3040889
5,2018,2,85,3040889
6,2019,1,59,3060508
7,2019,2,46,3060508
8,2020,1,31,3080306
9,2020,2,56,3080306


In [419]:
# agregamos una columna que contenga la tasa de homicidios
data_kpi01['TASA']= (data_kpi01['N_VICTIMAS'] / data_kpi01['POBLACION_TOTAL']) * 100000
data_kpi01

,AÑO,SEMESTRE,N_VICTIMAS,POBLACION_TOTAL,TASA
0,2016,1,67,3002178,2.231713
1,2016,2,82,3002178,2.731350
2,2017,1,81,3021444,2.680837
3,2017,2,79,3021444,2.614644
4,2018,1,76,3040889,2.499269
5,2018,2,85,3040889,2.795235
6,2019,1,59,3060508,1.927785
7,2019,2,46,3060508,1.503018
8,2020,1,31,3080306,1.006394
9,2020,2,56,3080306,1.818001


In [420]:
# ahora agregamos otra columna con la tasa del periodo anterior
data_kpi01['TASA_ANTERIOR']= data_kpi01['TASA'].shift(periods=1, fill_value=0)
data_kpi01

,AÑO,SEMESTRE,N_VICTIMAS,POBLACION_TOTAL,TASA,TASA_ANTERIOR
0,2016,1,67,3002178,2.231713,0.000000
1,2016,2,82,3002178,2.731350,2.231713
2,2017,1,81,3021444,2.680837,2.731350
3,2017,2,79,3021444,2.614644,2.680837
4,2018,1,76,3040889,2.499269,2.614644
5,2018,2,85,3040889,2.795235,2.499269
6,2019,1,59,3060508,1.927785,2.795235
7,2019,2,46,3060508,1.503018,1.927785
8,2020,1,31,3080306,1.006394,1.503018
9,2020,2,56,3080306,1.818001,1.006394


ahora procederemos a crear una columna con la variacion de la tasa, haciendo el calculo entre la columna 'TASA' y 'TASA ANTERIOR'

In [421]:
# creamos columna
data_kpi01['VARIACION_TASA']= (data_kpi01['TASA'] - data_kpi01['TASA_ANTERIOR']) / data_kpi01['TASA_ANTERIOR'] * 100
data_kpi01

,AÑO,SEMESTRE,N_VICTIMAS,POBLACION_TOTAL,TASA,TASA_ANTERIOR,VARIACION_TASA
0,2016,1,67,3002178,2.231713,0.000000,inf
1,2016,2,82,3002178,2.731350,2.231713,22.388060
2,2017,1,81,3021444,2.680837,2.731350,-1.849378
3,2017,2,79,3021444,2.614644,2.680837,-2.469136
4,2018,1,76,3040889,2.499269,2.614644,-4.412637
5,2018,2,85,3040889,2.795235,2.499269,11.842105
6,2019,1,59,3060508,1.927785,2.795235,-31.033191
7,2019,2,46,3060508,1.503018,1.927785,-22.033898
8,2020,1,31,3080306,1.006394,1.503018,-33.041839
9,2020,2,56,3080306,1.818001,1.006394,80.645161


In [422]:
data_kpi01["VARIACION_TASA"] = data_kpi01["VARIACION_TASA"].replace([np.inf, -np.inf], 0)
data_kpi01["VARIACION_TASA"] = round(data_kpi01["VARIACION_TASA"], 2)
data_kpi01

,AÑO,SEMESTRE,N_VICTIMAS,POBLACION_TOTAL,TASA,TASA_ANTERIOR,VARIACION_TASA
0,2016,1,67,3002178,2.231713,0.000000,0.00
1,2016,2,82,3002178,2.731350,2.231713,22.39
2,2017,1,81,3021444,2.680837,2.731350,-1.85
3,2017,2,79,3021444,2.614644,2.680837,-2.47
4,2018,1,76,3040889,2.499269,2.614644,-4.41
5,2018,2,85,3040889,2.795235,2.499269,11.84
6,2019,1,59,3060508,1.927785,2.795235,-31.03
7,2019,2,46,3060508,1.503018,1.927785,-22.03
8,2020,1,31,3080306,1.006394,1.503018,-33.04
9,2020,2,56,3080306,1.818001,1.006394,80.65


# KPI 2

Empezamos a construir la data necesaria para nuestro segundo kpi

Para calcular el segundo KPi necesitamos obtener la cantidad de muertes de motociclistas ocurridas, en cada año asi que procedemos a filtrar el dataframe df_homicidios

In [423]:
data_kpi02= df_homicidios[df_homicidios['VICTIMA'] == 'MOTO']
data_kpi02.head()

,ID,N_VICTIMAS,FECHA,HORA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,COORDENADAS_X,COORDENADAS_Y,PARTICIPANTES,VICTIMA,ACUSADO,AÑO,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO,SEMESTRE
0,2016-0001,1,2016-01-01,04:00:00,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,2016,CONDUCTOR,MASCULINO,19,2016-01-01,1
2,2016-0003,1,2016-01-03,07:00:00,AV ENTRE RIOS 2034,AVENIDA,1,-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,2016,CONDUCTOR,MASCULINO,30,2016-01-03,1
3,2016-0004,1,2016-01-10,00:00:00,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,8,-58.465039,-34.680930,MOTO-SD,MOTO,NO IDENTIFICADO,2016,CONDUCTOR,MASCULINO,18,2016-01-10,1
4,2016-0005,1,2016-01-21,05:20:00,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,1,-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,2016,CONDUCTOR,MASCULINO,29,2016-02-01,1
5,2016-0008,1,2016-01-24,18:30:00,AV 27 DE FEBRERO Y AV ESCALADA,AVENIDA,8,-58.444513,-34.684759,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO,2016,CONDUCTOR,MASCULINO,30,2016-01-24,1


In [424]:
# agrupamos por año y obtenemos el total de victimas en accidentes de motos anuales
data_kpi02 = data_kpi02.groupby(["AÑO"])["N_VICTIMAS"].sum().reset_index()
data_kpi02

,AÑO,N_VICTIMAS
0,2016,67
1,2017,62
2,2018,61
3,2019,49
4,2020,30
5,2021,46


In [425]:
# creamos una columna con el numero de victimas de años anteriores
data_kpi02['N_VICTIMAS_ANTERIORES']= data_kpi02['N_VICTIMAS'].shift(periods=1, fill_value=0)
data_kpi02

,AÑO,N_VICTIMAS,N_VICTIMAS_ANTERIORES
0,2016,67,0
1,2017,62,67
2,2018,61,62
3,2019,49,61
4,2020,30,49
5,2021,46,30


procedemos a calcular la variacion porcentual en el numero de victimas de accidentes de motos en comparacion con el año anterior y lo almacenaremos en una nueva columna llamada 'VARIACION_VICTIMAS_MOTO'

In [426]:
# hacemos el calculo y creamos columna
data_kpi02['VARIACION_VICTIMAS_MOTO'] = (data_kpi02['N_VICTIMAS'] - data_kpi02['N_VICTIMAS_ANTERIORES']) / data_kpi02['N_VICTIMAS_ANTERIORES'] * 100
data_kpi02

,AÑO,N_VICTIMAS,N_VICTIMAS_ANTERIORES,VARIACION_VICTIMAS_MOTO
0,2016,67,0,inf
1,2017,62,67,-7.462687
2,2018,61,62,-1.612903
3,2019,49,61,-19.672131
4,2020,30,49,-38.775510
5,2021,46,30,53.333333


In [427]:
# reemplazamos los valores infinitos y redondeamos la columna a dos decimales
data_kpi02["VARIACION_VICTIMAS_MOTO"] = data_kpi02["VARIACION_VICTIMAS_MOTO"].replace([np.inf, -np.inf], 0)
data_kpi02["VARIACION_VICTIMAS_MOTO"] = round(data_kpi02["VARIACION_VICTIMAS_MOTO"], 2)
data_kpi02

,AÑO,N_VICTIMAS,N_VICTIMAS_ANTERIORES,VARIACION_VICTIMAS_MOTO
0,2016,67,0,0.00
1,2017,62,67,-7.46
2,2018,61,62,-1.61
3,2019,49,61,-19.67
4,2020,30,49,-38.78
5,2021,46,30,53.33


# guardamos archivos csv



In [428]:
df_homicidios.to_csv('data/df_homicidios.csv')
data_kpi01.to_csv('data/data_kpi01.csv')
data_kpi02.to_csv('data/data_kpi02.csv')